# Pagination Controls

> Pattern for navigation between pages of content with HTMX integration

In [ ]:
#| default_exp patterns.pagination

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Callable, Any
from enum import Enum
from fasthtml.common import *

from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles, btn_behaviors
from cjm_fasthtml_daisyui.components.navigation.pagination import join
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, justify
from cjm_fasthtml_tailwind.core.base import combine_classes

## Pagination Style Options

The `PaginationStyle` enum defines different pagination display styles.

In [ ]:
#| export
class PaginationStyle(Enum):
    """Display styles for pagination controls."""
    SIMPLE = "simple"        # Previous/Next buttons with page info
    COMPACT = "compact"      # Previous/Next buttons only (no page info)
    NUMBERED = "numbered"    # Include numbered page buttons (future enhancement)

## PaginationControls Function

The `PaginationControls` function creates navigation controls for paginated content.

Key features:
- Previous/Next navigation buttons
- Current page and total pages display
- Automatic disabled states for boundary pages
- HTMX integration for SPA-like navigation
- URL management with `hx-push-url`
- Flexible route generation via callback
- DaisyUI button group styling
- Customizable button text and styling

In [ ]:
#| export
def PaginationControls(
    current_page: int,  # Current page number (1-indexed)
    total_pages: int,  # Total number of pages
    route_func: Callable[[int], str],  # Function to generate route for a page number
    target_id: str,  # HTML ID of element to update with HTMX
    style: PaginationStyle = PaginationStyle.SIMPLE,  # Pagination display style
    swap: str = "outerHTML",  # HTMX swap strategy (outerHTML, innerHTML, beforeend, etc.)
    prev_text: str = "« Previous",  # Text for previous button
    next_text: str = "Next »",  # Text for next button
    page_info_format: str = "Page {current} of {total}",  # Format string for page info
    button_size: str = None,  # Button size class (e.g., btn_sizes.sm)
    container_cls: Optional[str] = None,  # Additional classes for container
    push_url: bool = True,  # Whether to update URL with hx-push-url
    **kwargs  # Additional attributes for the container
) -> FT:  # Div element with pagination controls
    """Create pagination navigation controls with HTMX integration.
    
    The controls provide Previous/Next navigation with automatic disabled states
    at page boundaries. Uses HTMX for SPA-like page transitions without full
    page reloads.
    
    Examples:
        # Simple pagination for media library
        PaginationControls(
            current_page=5,
            total_pages=20,
            route_func=lambda p: f"/library?page={p}",
            target_id="library-content"
        )
        
        # With custom styling and text
        PaginationControls(
            current_page=1,
            total_pages=10,
            route_func=lambda p: media_rt.library.to(page=p, view="grid"),
            target_id=HtmlIds.MAIN_CONTENT,
            prev_text="← Back",
            next_text="Forward →",
            button_size=str(btn_sizes.sm)
        )
        
        # Compact style without page info
        PaginationControls(
            current_page=3,
            total_pages=8,
            route_func=lambda p: f"/results?page={p}",
            target_id="results",
            style=PaginationStyle.COMPACT
        )
        
        # Without URL updates (for modals/drawers)
        PaginationControls(
            current_page=2,
            total_pages=5,
            route_func=lambda p: f"/api/items?page={p}",
            target_id="modal-content",
            push_url=False
        )
        
        # With innerHTML swap (replaces content inside target)
        PaginationControls(
            current_page=3,
            total_pages=10,
            route_func=lambda p: f"/items?page={p}",
            target_id="items-container",
            swap="innerHTML"
        )
    """
    # Calculate prev/next page numbers
    prev_page = max(1, current_page - 1)
    next_page = min(total_pages, current_page + 1)
    
    # Check if we're at boundaries
    is_first_page = current_page <= 1
    is_last_page = current_page >= total_pages
    
    # Build button classes
    button_classes = [btn]
    if button_size:
        button_classes.append(button_size)
    
    # Create Previous button
    prev_button = A(
        prev_text,
        href=route_func(prev_page),
        hx_get=route_func(prev_page),
        hx_target=InteractionHtmlIds.as_selector(target_id),
        hx_swap=swap,
        hx_push_url="true" if push_url else None,
        cls=combine_classes(
            *button_classes,
            btn_styles.ghost if not is_first_page else btn_behaviors.disabled
        ),
        disabled=is_first_page
    )
    
    # Create Next button
    next_button = A(
        next_text,
        href=route_func(next_page),
        hx_get=route_func(next_page),
        hx_target=InteractionHtmlIds.as_selector(target_id),
        hx_swap=swap,
        hx_push_url="true" if push_url else None,
        cls=combine_classes(
            *button_classes,
            btn_styles.ghost if not is_last_page else btn_behaviors.disabled
        ),
        disabled=is_last_page
    )
    
    # Build button group content
    button_group_parts = [prev_button]
    
    # Add page info for SIMPLE style
    if style == PaginationStyle.SIMPLE:
        page_info = page_info_format.format(
            current=current_page,
            total=total_pages
        )
        button_group_parts.append(
            Span(page_info, cls=str(m.x(4)))
        )
    
    button_group_parts.append(next_button)
    
    # Build container classes
    container_classes = [
        flex_display,
        justify.center,
        m.t(6)
    ]
    if container_cls:
        container_classes.append(container_cls)
    
    # Create the pagination container
    return Div(
        Div(
            *button_group_parts,
            cls=str(join)
        ),
        cls=combine_classes(*container_classes),
        **kwargs
    )

## Usage Examples

Here are complete examples showing different pagination use cases:

In [ ]:
from cjm_fasthtml_daisyui.components.actions.button import btn_sizes

# Example 1: Simple pagination for media library
simple_pagination = PaginationControls(
    current_page=5,
    total_pages=20,
    route_func=lambda p: f"/library?page={p}",
    target_id="library-content"
)

print("Example 1: Simple pagination (page 5 of 20)")
print(f"  Previous button disabled: {5 <= 1}")
print(f"  Next button disabled: {5 >= 20}")
print(f"  Shows page info: True")

Example 1: Simple pagination (page 5 of 20)
  Previous button disabled: False
  Next button disabled: False
  Shows page info: True


In [ ]:
# Example 2: First page (Previous disabled)
first_page_pagination = PaginationControls(
    current_page=1,
    total_pages=10,
    route_func=lambda p: f"/results?page={p}",
    target_id="results-container"
)

print("Example 2: First page (Previous disabled)")
print(f"  Current page: 1")
print(f"  Previous button disabled: True")
print(f"  Next button disabled: False")

Example 2: First page (Previous disabled)
  Current page: 1
  Previous button disabled: True
  Next button disabled: False


In [ ]:
# Example 3: Last page (Next disabled)
last_page_pagination = PaginationControls(
    current_page=10,
    total_pages=10,
    route_func=lambda p: f"/results?page={p}",
    target_id="results-container"
)

print("Example 3: Last page (Next disabled)")
print(f"  Current page: 10")
print(f"  Previous button disabled: False")
print(f"  Next button disabled: True")

Example 3: Last page (Next disabled)
  Current page: 10
  Previous button disabled: False
  Next button disabled: True


In [ ]:
# Example 4: Compact style without page info
compact_pagination = PaginationControls(
    current_page=3,
    total_pages=8,
    route_func=lambda p: f"/items?page={p}",
    target_id="items-list",
    style=PaginationStyle.COMPACT
)

print("Example 4: Compact style")
print(f"  Shows page info: False")
print(f"  Only Previous/Next buttons")

Example 4: Compact style
  Shows page info: False
  Only Previous/Next buttons


In [ ]:
# Example 5: Custom button text and size
custom_pagination = PaginationControls(
    current_page=2,
    total_pages=5,
    route_func=lambda p: f"/search?q=test&page={p}",
    target_id="search-results",
    prev_text="← Back",
    next_text="Forward →",
    button_size=str(btn_sizes.sm),
    page_info_format="{current}/{total}"
)

print("Example 5: Custom styling")
print(f"  Button text: '← Back' / 'Forward →'")
print(f"  Page info format: '2/5'")
print(f"  Button size: small")

Example 5: Custom styling
  Button text: '← Back' / 'Forward →'
  Page info format: '2/5'
  Button size: small


In [ ]:
# Example 6: Without URL updates (for modals)
modal_pagination = PaginationControls(
    current_page=1,
    total_pages=3,
    route_func=lambda p: f"/api/modal-items?page={p}",
    target_id="modal-content",
    push_url=False
)

print("Example 6: Modal pagination (no URL push)")
print(f"  Updates URL: False")
print(f"  Target: modal-content")

Example 6: Modal pagination (no URL push)
  Updates URL: False
  Target: modal-content


## Server-Side Implementation

The server endpoint should return the paginated content that will replace the target container:

```python
@app.get("/library")
def library(page: int = 1):
    # Load data for current page
    items_per_page = 20
    total_items = get_total_items()
    total_pages = (total_items + items_per_page - 1) // items_per_page
    
    # Get items for current page
    start_idx = (page - 1) * items_per_page
    items = get_items(start=start_idx, limit=items_per_page)
    
    # Render content with pagination
    return Div(
        # Your content grid/list
        render_items(items),
        
        # Pagination controls
        PaginationControls(
            current_page=page,
            total_pages=total_pages,
            route_func=lambda p: f"/library?page={p}",
            target_id="library-content"
        ),
        
        id="library-content"
    )
```

## Complex Route Function

For pages with multiple query parameters, the route function can preserve all state:

```python
# Preserve view mode, filters, and search query
def make_route(page: int) -> str:
    return media_rt.library.to(
        page=page,
        view=view_mode,
        media_type=media_type,
        search=search_query
    )

pagination = PaginationControls(
    current_page=current_page,
    total_pages=total_pages,
    route_func=make_route,
    target_id="library-content"
)
```

## HTMX Integration Details

The pagination controls use HTMX attributes for SPA-like navigation:

- `hx-get`: Fetches the new page content
- `hx-target`: Specifies which element to update
- `hx-swap`: Determines how content is swapped (default: `outerHTML`)
- `hx-push-url`: Updates browser URL (optional)
- `href`: Fallback for non-JavaScript navigation

### Swap Strategies

The `swap` parameter controls how HTMX replaces content:

- **`outerHTML`** (default): Replaces the target element entirely. Use when your route returns content with the same ID as the target.
- **`innerHTML`**: Replaces content inside the target element. Use when the target container persists and only inner content changes.
- **`beforeend`**: Appends content to the end of the target. Useful for infinite scroll.
- **`afterbegin`**: Prepends content to the beginning of the target.

Example with `outerHTML` (default):
```python
# Route returns: Div(..., id="library-content")
PaginationControls(
    current_page=page,
    total_pages=total_pages,
    route_func=lambda p: f"/library?page={p}",
    target_id="library-content",
    swap="outerHTML"  # Replaces entire div
)
```

Example with `innerHTML`:
```python
# Route returns: content without wrapping div
PaginationControls(
    current_page=page,
    total_pages=total_pages,
    route_func=lambda p: f"/library/content?page={p}",
    target_id="library-container",
    swap="innerHTML"  # Replaces only inner content
)
```

### Disabled States

Disabled states are automatically handled:
- Previous button disabled on page 1
- Next button disabled on last page
- Uses DaisyUI `btn-disabled` styling

The pagination pattern works seamlessly with other HTMX features like indicators and loading states.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()